# <font color='green'>**Seguridad En Ruta: Desglose de Incidentes Víales por C5 (2022-2023)**</font>





## **Introducción**

En el marco del Día Mundial en Recuerdo de las Víctimas de Accidentes de Tránsito, el Instituto Nacional de Estadística y Geografía (INEGI) ha revelado cifras impactantes acerca de la magnitud de los accidentes de tránsito en México durante el año 2021, en el comunicado de prensa titulado *'ESTADÍSTICAS A PROPÓSITO DEL DÍA MUNDIAL EN RECUERDO DE LAS VÍCTIMAS DE ACCIDENTES DE TRÁNSITO'* (*INEGI, 2022*) se reportó un un total de 340,415 incidentes reportados a nivel nacional, que resultaron en 3,849 vidas perdidas y 60,584 personas lesionadas. Lo anterior resalta la la urgencia de abordar la seguridad vial de manera contundente.  
<br>
En respuesta a este panorama, hemos dirigido nuestra atención hacia la Ciudad de México, utilizando como referencia los datos proporcionados por el Centro de Comando, Control, Cómputo, Comunicaciones y Contacto Ciudadano de la Ciudad de México (C5). Este órgano gubernamental se presenta como el canal principal, siendo el primero en recibir reportes de incidentes viales.   
<br>
El propósito de este proyecto es llevar a cabo un análisis exhaustivo de las características de los incidentes viales en la Ciudad de México, iluminando patrones, tendencias y áreas potenciales de mejora en seguridad vial. Con la información detallada proporcionada por el C5 y la SSC, buscamos contribuir a la implementación de estrategias y medidas efectivas que reduzcan la incidencia de accidentes y salvaguarden la vida de los ciudadanos en las vías de la ciudad.

## **Objetivo**

<font color='red'>Nota: Escribir objetivo</font>  
El dataset utilizado en el desarrollo de este proyecto, denominado **'Incidentes Viales (2022-2023)'**, fue obtenido del **'Portal de Datos Abiertos: Sistema Ajolote'** del Gobierno de la Ciudad de México. Se trata de un conjunto de datos proporcionados por el C5, que puede ser encontrado en el siguiente link:  
[Incidentes viales reportados por C5 ](https://datos.cdmx.gob.mx/dataset/incidentes-viales-c5)



## **Análisis Exploratorio**

### Librerías

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### Carga de Datos

Para la carga de datos desde el dataset en formato CSV, se utiliza la función <font color='green'>**cargar_csv**</font>, la cual toma una URL como parámetro, asumiendo que el archivo CSV está alojado en la web, específicamente en un repositorio de GitHub. La función está diseñada con un bloque try/except: en el try, se asigna el resultado del método **read_csv** de la biblioteca Pandas a un objeto llamado df, y se imprime un mensaje indicando *'Carga de datos con éxito'*. En caso de que la operación no sea exitosa, se maneja la excepción y se imprime un mensaje de error.

Posterior a la carga del CSV, se imprimen los primeros 5 registros del DataFrame mediante el método **.head()**. En caso contrario, se imprime un mensaje de error.

In [15]:
def cargar_csv(url):
  try:
    df = pd.read_csv(url)
    print('Carga de datos con éxito.')
    return df
  except Exception as e:
    print(f'Error al leer el archivo CSV desde {url}: {e}');
    return None

In [16]:
url = 'https://raw.githubusercontent.com/DaniaGarciaM/SeguridadEnRuta-DS/main/inViales_2022_2023.csv'
df = cargar_csv(url)
df.head()

Carga de datos con éxito.


,folio,fecha_creacion,hora_creacion,dia_semana,fecha_cierre,hora_cierre,tipo_incidente_c4,incidente_c4,alcaldia_inicio,latitud,longitud,codigo_cierre,clas_con_f_alarma,tipo_entrada,alcaldia_cierre,colonia
0,C2C/20211229/00212,2021-12-29,23:21:20,Miércoles,2022-01-01,00:24:06,Lesionado,Atropellado,NaN,19.421610,-99.163670,I,URGENCIAS MEDICAS,BOTÓN DE AUXILIO,NaN,ROMA NORTE I
1,C2C/20211231/00183,2021-12-31,23:48:03,Viernes,2022-01-01,06:29:52,Lesionado,Atropellado,CUAUHTEMOC,19.409910,-99.144270,A,URGENCIAS MEDICAS,BOTÓN DE AUXILIO,CUAUHTEMOC,OBRERA III
2,C2C/20220101/00012,2022-01-01,01:06:39,Sábado,2022-01-01,06:00:12,Accidente,Choque con lesionados,CUAUHTEMOC,19.415250,-99.148320,A,URGENCIAS MEDICAS,BOTÓN DE AUXILIO,CUAUHTEMOC,DOCTORES IV
3,C2C/20220101/00070,2022-01-01,09:51:53,Sábado,2022-01-01,12:54:10,Accidente,Motociclista,CUAUHTEMOC,19.443649,-99.165781,A,URGENCIAS MEDICAS,RADIO,CUAUHTEMOC,TLAXPANA
4,C2N/20211231/00128,2021-12-31,23:03:05,Viernes,2022-01-01,06:16:42,Accidente,Choque sin lesionados,IZTACALCO,19.414620,-99.060450,A,EMERGENCIA,BOTÓN DE AUXILIO,IZTACALCO,PANTITLAN II
